In [3]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("music.sqlite")
cur = conn.cursor()
temporaryX = cur.execute("SELECT * FROM genres").fetchall()
df = pd.DataFrame(temporaryX)

In [4]:
df.head()

,0,1
0,22703,electronic
1,22721,metal
2,22659,rock
3,22661,rock
4,22725,electronic
